In [ ]:
!pip install ensemble_boxes

In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
submission_files = [
    '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/3/submission_latest.csv',
    '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_swin_t/0/submission_latest.csv',
    '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_swin_t/1/submission_latest.csv',
    '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/4/submission_latest.csv'
]
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [29]:
submission_df[3]['PredictionString'].map(zz).sort_values()

0       0
3250    0
3249    0
3248    0
3247    0
       ..
1620    0
1619    0
1618    0
1625    0
4870    0
Name: PredictionString, Length: 4871, dtype: int64

In [31]:
x= [1,2,3,4]
if x>4:
    print("zz")

TypeError: '>' not supported between instances of 'list' and 'int'

In [19]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.4

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        #boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=0.55, skip_box_thr=0.1)
        #boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, iou_thr=0.55,sigma=0.3, thresh=1e-4)
        
        
        for box, score, label in zip(boxes, scores, labels):
            score = round(score,6)
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(round(box[0] * image_info['width'],3)) + ' ' + str(round(box[1] * image_info['height'],3)) + ' ' + str(round(box[2] * image_info['width'],3)) + ' ' + str(round(box[3] * image_info['height'],3)) + ' '
        
        # for box, score, label in zip(boxes, scores, labels):
        #     prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [20]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/ensemble/swinconv2.csv',index=False)

submission.head()

,PredictionString,image_id
0,7 0.999875 603.008 516.393 950.567 1022.391 7 ...,test/0000.jpg
1,5 0.981224 132.021 1.091 497.412 286.71 6 0.94...,test/0001.jpg
2,1 0.969333 783.372 411.265 1023.672 1024.0 1 0...,test/0002.jpg
3,9 0.999629 151.291 263.211 912.516 828.255 9 0...,test/0003.jpg
4,1 0.981351 199.121 262.026 867.631 778.459 1 0...,test/0004.jpg


In [21]:
def zz(x):
    for i in x.split():
        if float(i) > 1024.0:
            return 1
    return 0
submission['PredictionString'].map(zz).sort_values()

0       0
3250    0
3249    0
3248    0
3247    0
       ..
1620    0
1619    0
1618    0
1625    0
4870    0
Name: PredictionString, Length: 4871, dtype: int64

In [14]:
zzstr = '1024.1111'
float(zzstr)

1024.1111

In [21]:
submission.loc[2300]['PredictionString']

'7.0 0.9998672008514404 0.0 45.32211685180664 1021.0397338867188 849.7864990234375 7.0 0.9950337409973145 354.26763916015625 2.936847686767578 1016.283935546875 405.2264709472656 1.0 0.9874979257583618 0.5636109709739685 800.7340698242188 470.8944396972656 1024.0001220703125 7.0 0.9872689247131348 0.0 1.0576039552688599 242.10284423828125 109.37150573730469 1.0 0.9544626474380493 171.81353759765625 0.32837405800819397 443.92694091796875 123.1866226196289 7.0 0.3792244791984558 353.1839904785156 0.4845025837421417 866.2257690429688 89.13346862792969 1.0 0.24694130569696426 193.41915893554688 2.5125346183776855 330.4344482421875 75.14266204833984 7.0 0.24102532863616943 194.82752990722656 3.2092232704162598 325.28533935546875 71.29825592041016 7.0 0.1905306875 834.1038 323.66968 1016.98004 472.35565 1.0 0.1149333125 2.444294 790.0718 58.659523 830.54956 1.0 0.1024992325 309.32626 82.376236 362.7903 133.52147 7.0 0.0786515325307846 448.1529541015625 1.4357051849365234 662.4606323242188 24